In [2]:
import torch
import torch.nn as nn
from torchvision import models # 다양한 학습 model을 포함하고 있는 module

# model download
# vgg16 - 사전 학습 모델
# https://pytorch.org/vision/stable/models/generated/torchvision.models.vgg16.html#vgg16
# VGG16_Weights.DEFAULT - 사전학습된 가중치, VGG16_Weights.IMAGENET1K_V1 과 동일
pretrained_model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
print(pretrained_model)

# 특성 추출기(features) 부분: Convolution layer(Conv2d)와 Pooling layer(MaxPool2d)로 구성
# Flatten(avgpool) 부분 - 2차원 이상의 tensor를 1차원 vector로 변환
# 분류기(classifier) - 1,000개의 label로 data를 분류 - 분류하려는 data에 맞게 수정 필요

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
class TransferLearningModel(nn.Module):

  def __init__(self, pretrained_model, feature_extractor):
    super().__init__()

    if (feature_extractor):
      for param in pretrained_model.parameters():
        param.require_grad = False

    # 학습 data에 맞게 새로운 classifier를 만든 후에,
    # 사전 학습 모델(Pre-Trained Model)의 classifier 부분을
    # 새로 만든 classifier로 변경
    pretrained_model.classifier = nn.Sequential(
        nn.Linear(pretrained_model.classifier[0].in_features, 128),
        nn.Linear(128, 2)
    )

    self.model = pretrained_model

  def forward(self, data):
    logits = self.model(data)
    return logits

feature_extractor = True # True: Feature Extractor,  False: Fine Tuning
model = TransferLearningModel(pretrained_model, feature_extractor)

criterion = nn.CrossEntropyLoss()
# 전이 학습(Transder Learning)의 Fine Tuning에서는
# 사전 학습 모델에서 사용된 가중치를 학습 data에 맞게 미세하게 조정해야 하므로
# 학습률(learning rate)을 lr=1e-6 으로 아주 적게 설정
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

print(model)

TransferLearningModel(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True